In [1]:
import pandas as pd
import json
from sklearn.utils import shuffle

In [2]:
with open("model_2_data.json", "r") as f:
    data = json.load(f)

id_names = []
X = []
Y = []

def norm(vote):
    if vote == 0:
        return (0, 1)
    elif vote == 1:
        return (1, 1)
    elif vote == 2:
        return (-1, 1)
    else:
        return (0, 0)

for person in data:
    id_names.append(
        {
            "id": person["id"],
            "first_name": person["first_name"],
            "last_name": person["last_name"],
        }
    )
    X.append([item for vote in person["votes"] for item in norm(vote)])
    Y.append(person["spider"])

    id_names, X, Y = shuffle(id_names, X, Y, random_state=42)

id_names = pd.DataFrame(id_names)
X = pd.DataFrame(X)
Y = pd.DataFrame(Y)

display(id_names.head())
display(X.head())
display(Y.head())

,id,first_name,last_name
0,..,..,..
1,..,..,..
2,..,..,..
3,..,..,..
4,..,..,..


,0,1,2,3,4,5
0,1,1,-1,1,0,0
1,1,1,-1,1,0,0
2,1,1,-1,1,0,0
3,1,1,-1,1,0,0
4,1,1,-1,1,0,0


,0,1,2,3
0,0.25,0.88,0.42,0.1312
1,0.25,0.88,0.42,0.1312
2,0.25,0.88,0.42,0.1312
3,0.25,0.88,0.42,0.1312
4,0.25,0.88,0.42,0.1312


In [3]:
import torch
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import torch.nn as nn

# Convert pandas DataFrames to numpy arrays
X_np = X.values.astype('float32')
Y_np = Y.values.astype('float32')

# Optionally scale X
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_np)

# Split into train and test sets
X_train, X_test, Y_train, Y_test = train_test_split(X_scaled, Y_np, test_size=0.2, random_state=42)

# Convert to torch tensors
X_train_tensor = torch.tensor(X_train)
Y_train_tensor = torch.tensor(Y_train)
X_test_tensor = torch.tensor(X_test)
Y_test_tensor = torch.tensor(Y_test)

# Define the neural network
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(X.shape[1], 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 16),
            nn.ReLU(),
            nn.Linear(16, Y.shape[1]),
            nn.Sigmoid()
        )
        self.net = nn.Sequential(
            nn.Linear(X.shape[1], 16),
            nn.ReLU(),
            nn.Linear(16, Y.shape[1])
        )
    def forward(self, x):
        return self.net(x)

model = SimpleNN()
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

# Training loop
epochs = 200
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train_tensor)
    loss = criterion(outputs, Y_train_tensor)
    loss.backward()
    optimizer.step()
    print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item():.4f}")

# Evaluate on test set
model.eval()
with torch.no_grad():
    test_outputs = model(X_test_tensor)
    test_loss = criterion(test_outputs, Y_test_tensor)
    print(f"Test Loss: {test_loss.item():.4f}")

Epoch 1/200, Loss: 0.3449
Epoch 2/200, Loss: 0.3067
Epoch 3/200, Loss: 0.2718
Epoch 4/200, Loss: 0.2397
Epoch 5/200, Loss: 0.2105
Epoch 6/200, Loss: 0.1842
Epoch 7/200, Loss: 0.1604
Epoch 8/200, Loss: 0.1388
Epoch 9/200, Loss: 0.1192
Epoch 10/200, Loss: 0.1015
Epoch 11/200, Loss: 0.0855
Epoch 12/200, Loss: 0.0711
Epoch 13/200, Loss: 0.0583
Epoch 14/200, Loss: 0.0471
Epoch 15/200, Loss: 0.0376
Epoch 16/200, Loss: 0.0297
Epoch 17/200, Loss: 0.0233
Epoch 18/200, Loss: 0.0182
Epoch 19/200, Loss: 0.0144
Epoch 20/200, Loss: 0.0116
Epoch 21/200, Loss: 0.0097
Epoch 22/200, Loss: 0.0083
Epoch 23/200, Loss: 0.0074
Epoch 24/200, Loss: 0.0067
Epoch 25/200, Loss: 0.0063
Epoch 26/200, Loss: 0.0060
Epoch 27/200, Loss: 0.0057
Epoch 28/200, Loss: 0.0056
Epoch 29/200, Loss: 0.0055
Epoch 30/200, Loss: 0.0055
Epoch 31/200, Loss: 0.0055
Epoch 32/200, Loss: 0.0055
Epoch 33/200, Loss: 0.0055
Epoch 34/200, Loss: 0.0053
Epoch 35/200, Loss: 0.0052
Epoch 36/200, Loss: 0.0049
Epoch 37/200, Loss: 0.0046
Epoch 38/2